# Organizing files, folders for sleuth 

Will be trying snakemake pipeline: https://github.com/snakemake-workflows/rna-seq-kallisto-sleuth/

Will use metadata.txt to build up the two config files: samples.tsv and units.tsv

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import bbi

Importing project folder and files for Liu_2021

In [2]:
proj = "/project/fudenber_735/GEO/liu_deWit_2021_GSE135180/pysradb_downloads/SRP216929"
meta = pd.read_csv(proj+"/metadata.tsv", sep="\t")
rnaseq = meta[meta['library_strategy'] == 'RNA-Seq']

# Get the AID-tagged samples by experiment title 
dWaplSamples = rnaseq[rnaseq['experiment_title'].str.contains('Wapl-C20') | rnaseq['experiment_title'].str.contains('PT')]

# Parental cell line for comparison
#normalSamples = rnaseq[rnaseq['experiment_title'].str.contains('PT')]

In [3]:
dWaplSamples

,Unnamed: 0,study_accession,experiment_accession,experiment_title,experiment_desc,organism_taxid,organism_name,library_name,library_strategy,library_source,...,sample_accession,sample_title,instrument,instrument_model,instrument_model_desc,total_spots,total_size,run_accession,run_total_spots,run_total_bases
26,26,SRP216929,SRX9106192,GSM4775363: 102_Wapl-C20_96h_RNA_rep2; Mus mus...,GSM4775363: 102_Wapl-C20_96h_RNA_rep2; Mus mus...,10090,Mus musculus,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SRS7349609,NaN,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,11899451,363500402,SRR12623461,11899451,773464315
27,27,SRP216929,SRX9106191,GSM4775362: 101_Wapl-C20_96h_RNA_rep1; Mus mus...,GSM4775362: 101_Wapl-C20_96h_RNA_rep1; Mus mus...,10090,Mus musculus,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SRS7349608,NaN,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,13067602,395757318,SRR12623460,13067602,849394130
28,28,SRP216929,SRX9106190,GSM4775361: 100_Wapl-C20_48h_RNA_rep2; Mus mus...,GSM4775361: 100_Wapl-C20_48h_RNA_rep2; Mus mus...,10090,Mus musculus,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SRS7349606,NaN,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,13484878,405798908,SRR12623459,13484878,876517070
29,29,SRP216929,SRX9106189,GSM4775360: 99_Wapl-C20_48h_RNA_rep1; Mus musc...,GSM4775360: 99_Wapl-C20_48h_RNA_rep1; Mus musc...,10090,Mus musculus,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SRS7349607,NaN,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,14334712,432965093,SRR12623458,14334712,931756280
30,30,SRP216929,SRX9106188,GSM4775359: 98_Wapl-C20_24h_RNA_rep2; Mus musc...,GSM4775359: 98_Wapl-C20_24h_RNA_rep2; Mus musc...,10090,Mus musculus,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SRS7349605,NaN,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,12549892,380208709,SRR12623457,12549892,815742980
31,31,SRP216929,SRX9106187,GSM4775358: 97_Wapl-C20_24h_RNA_rep1; Mus musc...,GSM4775358: 97_Wapl-C20_24h_RNA_rep1; Mus musc...,10090,Mus musculus,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SRS7349604,NaN,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,14446052,434692207,SRR12623456,14446052,938993380
32,32,SRP216929,SRX9106186,GSM4775357: 96_Wapl-C20_6h_RNA_rep2; Mus muscu...,GSM4775357: 96_Wapl-C20_6h_RNA_rep2; Mus muscu...,10090,Mus musculus,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SRS7349603,NaN,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,13946514,420967912,SRR12623455,13946514,906523410
33,33,SRP216929,SRX9106185,GSM4775356: 95_Wapl-C20_6h_RNA_rep1; Mus muscu...,GSM4775356: 95_Wapl-C20_6h_RNA_rep1; Mus muscu...,10090,Mus musculus,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SRS7349601,NaN,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,14623696,441425307,SRR12623454,14623696,950540240
34,34,SRP216929,SRX9106184,GSM4775355: 94_Wapl-C20_0h_RNA_rep2; Mus muscu...,GSM4775355: 94_Wapl-C20_0h_RNA_rep2; Mus muscu...,10090,Mus musculus,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SRS7349602,NaN,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,13739771,414117159,SRR12623453,13739771,893085115
35,35,SRP216929,SRX9106183,GSM4775354: 93_Wapl-C20_0h_RNA_rep1; Mus muscu...,GSM4775354: 93_Wapl-C20_0h_RNA_rep1; Mus muscu...,10090,Mus musculus,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SRS7349600,NaN,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,15272415,459754694,SRR12623452,15272415,992706975


Need to set up configuration files for the snakemake workflow. Importing example files, will build our own and re-name them:

In [4]:
config_files = "/home1/pokorny/rna-seq-kallisto-sleuth-workflow/config"
config_sample = pd.read_csv(config_files+"/samples.tsv", sep = "\t")
config_units = pd.read_csv(config_files+"/units.tsv", sep="\t")

In [5]:
## will not include batch for first round -- technical replicates should just be pooled by kallisto 
config_sample

,Unnamed: 0,sample,condition
0,0,PT_0h,PT_0h
1,1,PT_96h,PT_96h
2,2,Wapl-C20_0h,Wapl-C20_0h
3,3,Wapl-C20_24h,Wapl-C20_24h
4,4,Wapl-C20_48h,Wapl-C20_48h
5,5,Wapl-C20_6h,Wapl-C20_6h
6,6,Wapl-C20_96h,Wapl-C20_96h


In [6]:
## fq1 is proj + 'experiment_accession' + 'run_accession' .fastq
## unit = rep[#]
## sample = 'experiment_title'.split(_)
## all fragment_len_mean = 65 
# Note: cannot find fragment_len_sd because cannot find it (maybe ask/look harder later?)

config_units

,Unnamed: 0,sample,unit,fragment_len_mean,fragment_len_sd,fq1,fq2
0,0,PT_0h,2,65,1.5,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,NaN
1,1,PT_0h,1,65,1.5,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,NaN
2,2,PT_96h,2,65,1.5,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,NaN
3,3,PT_96h,1,65,1.5,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,NaN
4,4,Wapl-C20_0h,2,65,1.5,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,NaN
5,5,Wapl-C20_0h,1,65,1.5,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,NaN
6,6,Wapl-C20_24h,2,65,1.5,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,NaN
7,7,Wapl-C20_24h,1,65,1.5,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,NaN
8,8,Wapl-C20_48h,2,65,1.5,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,NaN
9,9,Wapl-C20_48h,1,65,1.5,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,NaN


In [7]:
# Parental cell line for comparison
#dWaplSamples['condition'] = dWaplSamples['experiment_title'].apply(lambda x: x.split(r'[0-9]_')[1].split('_RNA')[0])

def extractConfig(experiment_string):
    
    #experiment_string = ['GSM4775363: 102_Wapl-C20_96h_RNA_rep2; Mus mus']
    hi = experiment_string.split('_')
    condition = hi[1]+'_'+hi[2]
    rep = hi[4].split('rep')[1].split(';')[0]
    
    return [condition, rep]

def buildFastaLoc(exp, run):
    
    return proj + "/" + str(exp) + "/" + str(run) + ".fastq"
    

In [8]:
dWaplSamples['condition_unit'] = dWaplSamples.loc[:,('experiment_title')].apply(lambda x: extractConfig(str(x)))

/tmp/SLURM_4169580/ipykernel_14164/523196690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dWaplSamples['condition_unit'] = dWaplSamples.loc[:,('experiment_title')].apply(lambda x: extractConfig(str(x)))


In [9]:
experiment_string = 'GSM4775363: 102_Wapl-C20_96h_RNA_rep2; Mus mus'

print(extractConfig(experiment_string))

['Wapl-C20_96h', '2']


In [10]:
dWaplSamples['condition_unit'] = dWaplSamples.loc[:,('experiment_title')].apply(lambda x: extractConfig(str(x)))

# splitting the two returned values into two columns
config_df = pd.DataFrame(dWaplSamples['condition_unit'].to_list(), columns=['sample', 'unit'])

/tmp/SLURM_4169580/ipykernel_14164/1352354544.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dWaplSamples['condition_unit'] = dWaplSamples.loc[:,('experiment_title')].apply(lambda x: extractConfig(str(x)))


In [12]:
# Adding the fastq file locations
fq1 = dWaplSamples['experiment_accession'].copy()
fq1 = [buildFastaLoc(x, y) for x,y in zip(dWaplSamples['experiment_accession'], dWaplSamples['run_accession'])]

config_df['fq1'] = fq1

In [13]:
# Setting the rest of the values -- we have mean fragment length = 65 from illumina protocol. The rest idk
config_df['fragment_len_mean'] = 65

NaN = np.nan
config_df['fragment_len_sd'] = 1.5
config_df['fq2'] = NaN


In [14]:
config_df

,sample,unit,fq1,fragment_len_mean,fragment_len_sd,fq2
0,Wapl-C20_96h,2,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,65,1.5,NaN
1,Wapl-C20_96h,1,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,65,1.5,NaN
2,Wapl-C20_48h,2,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,65,1.5,NaN
3,Wapl-C20_48h,1,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,65,1.5,NaN
4,Wapl-C20_24h,2,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,65,1.5,NaN
5,Wapl-C20_24h,1,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,65,1.5,NaN
6,Wapl-C20_6h,2,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,65,1.5,NaN
7,Wapl-C20_6h,1,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,65,1.5,NaN
8,Wapl-C20_0h,2,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,65,1.5,NaN
9,Wapl-C20_0h,1,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,65,1.5,NaN


In [15]:
config_df = config_df.sort_values(by='sample', ignore_index=True)

In [16]:
# sample	unit	fragment_len_mean	fragment_len_sd	fq1	fq2

config_df = config_df.reindex(columns=['sample', 'unit', 'fragment_len_mean', 'fragment_len_sd', 'fq1', 'fq2'])

In [17]:
config_df['fq1'][0]

'/project/fudenber_735/GEO/liu_deWit_2021_GSE135180/pysradb_downloads/SRP216929/SRX6623249/SRR9870513.fastq'

In [18]:
list(lst), list(lst)

NameError: name 'lst' is not defined

In [19]:
lst = np.unique(config_df['sample'])

sample_df = pd.DataFrame({'sample' : lst, 
                         'condition' : lst})

In [20]:
config_sample

,Unnamed: 0,sample,condition
0,0,PT_0h,PT_0h
1,1,PT_96h,PT_96h
2,2,Wapl-C20_0h,Wapl-C20_0h
3,3,Wapl-C20_24h,Wapl-C20_24h
4,4,Wapl-C20_48h,Wapl-C20_48h
5,5,Wapl-C20_6h,Wapl-C20_6h
6,6,Wapl-C20_96h,Wapl-C20_96h


In [21]:
sample_df

,sample,condition
0,PT_0h,PT_0h
1,PT_96h,PT_96h
2,Wapl-C20_0h,Wapl-C20_0h
3,Wapl-C20_24h,Wapl-C20_24h
4,Wapl-C20_48h,Wapl-C20_48h
5,Wapl-C20_6h,Wapl-C20_6h
6,Wapl-C20_96h,Wapl-C20_96h


In [22]:
sample_df.to_csv(config_files+"/samples.tsv", sep = "\t")
config_df.to_csv(config_files+"/units.tsv", sep="\t")

In [23]:
# todo: finish formatting config_df and save to ../rna-seq-kallisto folder
# add WT/Parent to the files
# finish config_samples --> probably need to make sure that it's in the correct order for kallisto/sleuth
# run thing!

In [ ]:
# name sample and condition the same

# Running Sleuth the usual way

Already have sort of sample 2 condition from above, snakemake requires internet

In [41]:
def buildKallistoPath(exp):
    
    return proj + "/" + str(exp) + "/kallisto"

def extractCondition(experiment_string):
    
    #experiment_string = ['GSM4775363: 102_Wapl-C20_96h_RNA_rep2; Mus mus']
    hi = experiment_string.split('_')
    condition = hi[1]+'_'+hi[2]
    #rep = hi[4].split('rep')[1].split(';')[0]
    
    return condition

In [42]:
# Adding the fastq file locations
kallistofiles = dWaplSamples['experiment_accession'].copy()
kallistofiles = [[x, buildKallistoPath(y), extractCondition(z)] for x,y,z in zip(dWaplSamples['run_accession'], 
                                                                                 dWaplSamples['experiment_accession'], 
                                                                                 dWaplSamples['experiment_title'])]

In [43]:
print(kallistofiles)

[['SRR12623461', '/project/fudenber_735/GEO/liu_deWit_2021_GSE135180/pysradb_downloads/SRP216929/SRX9106192/kallisto', 'Wapl-C20_96h'], ['SRR12623460', '/project/fudenber_735/GEO/liu_deWit_2021_GSE135180/pysradb_downloads/SRP216929/SRX9106191/kallisto', 'Wapl-C20_96h'], ['SRR12623459', '/project/fudenber_735/GEO/liu_deWit_2021_GSE135180/pysradb_downloads/SRP216929/SRX9106190/kallisto', 'Wapl-C20_48h'], ['SRR12623458', '/project/fudenber_735/GEO/liu_deWit_2021_GSE135180/pysradb_downloads/SRP216929/SRX9106189/kallisto', 'Wapl-C20_48h'], ['SRR12623457', '/project/fudenber_735/GEO/liu_deWit_2021_GSE135180/pysradb_downloads/SRP216929/SRX9106188/kallisto', 'Wapl-C20_24h'], ['SRR12623456', '/project/fudenber_735/GEO/liu_deWit_2021_GSE135180/pysradb_downloads/SRP216929/SRX9106187/kallisto', 'Wapl-C20_24h'], ['SRR12623455', '/project/fudenber_735/GEO/liu_deWit_2021_GSE135180/pysradb_downloads/SRP216929/SRX9106186/kallisto', 'Wapl-C20_6h'], ['SRR12623454', '/project/fudenber_735/GEO/liu_deWit_20

In [45]:
# split the nested list into dictionary, then build dataframe
s2cDict = {'sample' : [],
           'condition' : [],
           'path' : []}
for entry in kallistofiles:
    s2cDict['sample'].append(entry[0])
    s2cDict['condition'].append(entry[1])
    s2cDict['path'].append(entry[2])

In [46]:
s2c = pd.DataFrame(s2cDict)

In [47]:
s2c

,sample,condition,path
0,SRR12623461,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,Wapl-C20_96h
1,SRR12623460,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,Wapl-C20_96h
2,SRR12623459,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,Wapl-C20_48h
3,SRR12623458,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,Wapl-C20_48h
4,SRR12623457,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,Wapl-C20_24h
5,SRR12623456,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,Wapl-C20_24h
6,SRR12623455,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,Wapl-C20_6h
7,SRR12623454,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,Wapl-C20_6h
8,SRR12623453,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,Wapl-C20_0h
9,SRR12623452,/project/fudenber_735/GEO/liu_deWit_2021_GSE13...,Wapl-C20_0h


In [49]:
s2c.to_csv(proj+'/s2c.tsv', sep='\t')

OSError: Cannot save file into a non-existent directory: '/project/fudenber_735/GEO/liu_deWit_2021_GSE135180/pysradb_downloads/SRP216929/RNA-seq'